In [2]:
#first we need to import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Input
import nltk
import re
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Nikhil
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Nikhil
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
#Now we need to import our training data which you can download through kaggle - https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/code

df=pd.read_csv("IMDB Dataset.csv")
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [5]:
#data's dimentions and other information

df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,negative
freq,5,25000


In [6]:
#checking if the data can be trusted , that is it is not much biased towards one side
print("postive ",np.sum(df["sentiment"]=="positive"))
print("negative ",np.sum(df["sentiment"]=="negative"))

postive  25000
negative  25000


In [7]:
#now begins the cleaning phase where we remove unneccessary letters and symbols from our data

def clear_noise(text):
  
  # removing html scripts using Beautiful Soup
  text1=BeautifulSoup(text)
  text1=text1.get_text()

  #removing all other character than alphabets and numbers using regular expression
  text2=re.sub('[^a-zA-Z0-9\s]', '', text1)

  return text2

df["review"]=df["review"].apply(clear_noise)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [8]:
# 1. Tokenisation (Breaking down the document into words)

def tokenize(text):
  return text.split()

# 2. Stopword Removal (Removal of words which are not meaningful for the model)

sw=set(nltk.corpus.stopwords.words('english'))

def stopword_removal(text):
  useful_words=[w for w in text if w not in sw]
  return useful_words

# 3. Lemmatization (Changing all forms of a verb to root form like plays ,played etc. to play)

wn=nltk.stem.WordNetLemmatizer()

def lemmatize(text):
  words=[wn.lemmatize(w) for w in text]
  return words

# 4. Building a vocabulary (Each sentence will have a feature vector)

def myTokenizer(text):
  text1=tokenize(text.lower())
  text1=stopword_removal(text1)
  text1=lemmatize(text1)
  return text1

cv=CountVectorizer(tokenizer=myTokenizer)

In [9]:
print(sw)

{'by', 'each', "mustn't", 'when', 'been', 'other', 'having', 'most', "don't", 'over', 'an', 'couldn', 'more', 'for', 'ain', 'above', 'd', 'these', 'those', 'should', 'do', 'or', 'doing', 'who', 'himself', 'where', "mightn't", 'own', 'how', 'it', "didn't", 'their', 'yourself', 'that', 'all', 'which', 'she', 'shan', "wasn't", 'then', 'while', 'him', 'too', 're', 'does', 'as', 'off', 'he', 'with', 'a', "aren't", 'under', 'yours', 'm', "hasn't", "weren't", 'whom', 'herself', 's', 'ma', 'had', 'here', 'shouldn', "shouldn't", "hadn't", 'we', "you've", 'this', 'll', 'there', 'his', 'no', "you'll", 'now', 'any', 'than', "should've", 'again', 'before', "haven't", 'my', 'until', 'through', "doesn't", 'aren', 'from', 'didn', 'both', 'once', 'needn', 'can', 'and', 'wasn', 'hasn', 'its', 'haven', 'have', 'against', 'are', 'after', 'hadn', 'ourselves', "isn't", 'don', 'is', 'myself', 'to', 'some', 'at', 'just', 'will', 'your', 'was', 'wouldn', 'being', 'not', 'theirs', 'but', 'weren', 'me', "wouldn'

In [10]:
#Now we need to split our data into training and testing. ideally the ratio should be around 80:20

train_x=df.review[:35000]
train_y=df.sentiment[:35000]

val_x=df.review[35000:40000]
val_y=df.sentiment[35000:40000]

test_x=df.review[40000:]
test_y=df.sentiment[40000:]

print(train_x.shape,type(train_x))
print(test_x.shape,type(test_x))

(35000,) <class 'pandas.core.series.Series'>
(10000,) <class 'pandas.core.series.Series'>


In [11]:
# Applying all the above operations on the training data

vectorised_data=cv.fit_transform(train_x)

In [12]:
print(vectorised_data.shape)

(35000, 166729)


In [13]:
# Converting the test and train data in vectorised form

cv_train=cv.transform(train_x)
cv_val=cv.transform(val_x)
cv_test=cv.transform(test_x)

In [14]:
print(cv_train.shape)
print(cv_val.shape)
print(cv_test.shape)

(35000, 166729)
(5000, 166729)
(10000, 166729)


In [15]:
# Labelling the sentiment data

lb=LabelBinarizer()
sentiment_data=lb.fit_transform(df['sentiment'])
print(sentiment_data.shape)

train_sentiments=sentiment_data[:35000]
val_sentiments=sentiment_data[35000:40000]
test_sentiments=sentiment_data[40000:]

(50000, 1)


In [16]:
# Building the MLP (Multi layer perceptron) Model
model=Sequential()
model.add(Input(shape=(166729,),sparse=True))
model.add(Dense(16,activation="relu"))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                2667680   
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 2,667,969
Trainable params: 2,667,969
Non-trainable params: 0
_________________________________________________________________


In [ ]:
hist=model.fit(cv_train,train_sentiments,epochs=50,batch_size=512,validation_data=(cv_val,val_sentiments))

Epoch 1/50
69/69 [==============================] - 2s 36ms/step - loss: 0.6835 - accuracy: 0.6253 - val_loss: 0.6665 - val_accuracy: 0.7066
Epoch 2/50
69/69 [==============================] - 2s 30ms/step - loss: 0.6454 - accuracy: 0.7488 - val_loss: 0.6275 - val_accuracy: 0.7596
Epoch 3/50
69/69 [==============================] - 2s 31ms/step - loss: 0.6055 - accuracy: 0.7869 - val_loss: 0.5885 - val_accuracy: 0.7854
Epoch 4/50
69/69 [==============================] - 2s 31ms/step - loss: 0.5655 - accuracy: 0.8042 - val_loss: 0.5498 - val_accuracy: 0.7944
Epoch 5/50
69/69 [==============================] - 2s 30ms/step - loss: 0.5264 - accuracy: 0.8160 - val_loss: 0.5124 - val_accuracy: 0.8126
Epoch 6/50
69/69 [==============================] - 2s 30ms/step - loss: 0.4905 - accuracy: 0.8274 - val_loss: 0.4793 - val_accuracy: 0.8212
Epoch 7/50
69/69 [==============================] - 2s 30ms/step - loss: 0.4592 - accuracy: 0.8365 - val_loss: 0.4513 - val_accuracy: 0.8340
Epoch 8/50
69

In [ ]:
#Now we need to check how our model performed by analysing the accuracy and loss after each iteration

In [ ]:
h=hist.history
plt.plot(h['val_loss'],label="validation loss")
plt.plot(h['loss'],label="training_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(h['accuracy'],label="training_accuracy")
plt.plot(h['val_accuracy'],label="val_accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
model.evaluate(cv_test,test_sentiments)

In [ ]:
#our model performed decent with an accuracy of around 90 %